In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma  # Updated import
from langchain.retrievers import BM25Retriever
from langchain.schema import Document
from typing import List, Optional
import pickle
from pathlib import Path
from tqdm import tqdm
import hashlib
import json
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ============================================================================
# CONFIGURATION
# ============================================================================
PATH = "../data/ISLP_website.pdf"
# CHROMA_DIR = '../stats/chroma_semantic'
# BM25_DIR = "../stats/bm25_retriever.pkl"
# METADATA_DIR = "../stats/doc_metadata.json"
# COLLECTION_NAME = "agentic_rag_collection"  # Explicit collection name

# ============================================================================
# EMBEDDINGS
# ============================================================================
embeddings = OllamaEmbeddings(model='nomic-embed-text')

# ============================================================================
# OPTIMIZED DOCUMENT LOADING
# ============================================================================
def load_documents(file_path: str) -> List[Document]:
    """
    Load documents with rich metadata (LangChain 0.3+ compatible)
    """
    logger.info(f"Loading documents from {file_path}")
    
    loader = PyMuPDFLoader(
        file_path=file_path,
        extract_images=True,
        extract_tables='markdown'
    )
    
    docs = loader.load()
    
    # Add rich metadata for better filtering
    for i, doc in enumerate(docs):
        doc.metadata.update({
            'page': i + 1,
            'source': file_path,
            'doc_id': hashlib.md5(doc.page_content.encode()).hexdigest()[:8],
            'file_name': Path(file_path).name,
            'total_pages': len(docs)
        })
    
    logger.info(f"✅ Loaded {len(docs)} pages")
    return docs

c:\Users\Dhaya\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
docs = load_documents(PATH)

INFO:__main__:Loading documents from ../data/ISLP_website.pdf


Consider using the pymupdf_layout package for a greatly improved page layout analysis.


INFO:__main__:✅ Loaded 613 pages


In [11]:
from rich import print
print(docs[450].page_content)

444
10. Deep Learning
In [32]: del(Hitters,
hit_model, hit_dm,
hit_logger,
hit_test, hit_train,
X, Y,
X_test, X_train,
Y_test, Y_train,
X_test_t, Y_test_t,
hit_trainer , hit_module)
10.9.2
Multilayer Network on the MNIST Digit Data
The torchvision package comes with a number of example datasets, includ-
ing the MNIST digit data. Our first step is to retrieve the training and test
data sets; the MNIST() function within torchvision.datasets is provided for MNIST()
this purpose. The data will be downloaded the first time this function is
executed, and stored in the directory data/MNIST.
In [33]: (mnist_train ,
mnist_test) = [MNIST(root='data',
train=train,
download=True,
transform=ToTensor())
for train in [True, False]]
mnist_train
Out[33]: Dataset MNIST
Number of datapoints: 60000
Root location: data
Split: Train
StandardTransform
Transform: ToTensor()
There are 60,000 images in the training data and 10,000 in the test data.
The images are 28 × 28, and stored as a matrix of pixels. We need to
transform each one into a vector.
Neural networks are somewhat sensitive to the scale of the inputs, much
as ridge and lasso regularization are affected by scaling. Here the inputs
are eight-bit grayscale values between 0 and 255, so we rescale to the unit
interval.26 This transformation, along with some reordering of the axes, is
performed by the ToTensor() transform from the torchvision.transforms
package.
As in our Hitters example, we form a data module from the training and
test datasets, setting aside 20% of the training images for validation.
In [34]: mnist_dm = SimpleDataModule(mnist_train ,
mnist_test ,
validation=0.2,
num_workers=max_num_workers ,
batch_size=256)
26Note: eight bits means 28, which equals 256. Since the convention is to start at 0,
the possible values range from 0 to 255.